In [6]:
from math import ceil
import json
import os
import sys
import random
from itertools import combinations


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from datasets import link_prediction
from layers import MeanAggregator, LSTMAggregator, MaxPoolAggregator, MeanPoolAggregator
from models import DGNN, AAGNN
from models_variants import EAAGNN, EAACGNN
import models
import utils

In [2]:
# Set up arguments for datasets, models and training.
is_train = True
is_test = False
is_val = True
is_debug = True
conf_device = None
hidden_dim = [8]
batch_size = 32
dataset_folder = "gen2_synthetic_csv_test_files/70_30_split"

num_layers = len(hidden_dim) + 1


if False and torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
conf_device = device

# Get the dataset, dataloader and model.
if not is_val and not is_test:
    dataset_args = ('train', num_layers)

if is_val:
    dataset_args = ('val', num_layers)

if is_test:
    dataset_args = ('test', num_layers)

datasets = utils.get_dataset_gcn(dataset_args, dataset_folder, is_debug=is_debug)


--------------------------------
Reading edge dataset from HE_T1087_84_Default_Extended_1_2


/Users/hasegawatai/college/research/DigitalPathology/Digital-Pathology/datasets/link_prediction.py:410: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distances_close_to_edges[target][source] = distance
/Users/hasegawatai/college/research/DigitalPathology/Digital-Pathology/datasets/link_prediction.py:409: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distances_close_to_edges[source][target] = distance


<<<<<<<<<<<<<         id  lym  epi  fib  inf     x     y                         gt  \
0        0  0.0  0.0  0.0  0.0   127  1682  apoptosis / civiatte body   
1        1  0.0  0.0  0.0  0.0   147  1690  apoptosis / civiatte body   
2        2  0.0  0.0  0.0  0.0   315  1656  apoptosis / civiatte body   
3        3  0.0  1.0  0.0  0.0  1027  1763                          3   
4        4  0.0  1.0  0.0  0.0  1122  1879                          3   
...    ...  ...  ...  ...  ...   ...   ...                        ...   
1008  1008  0.0  1.0  0.0  0.0   487  1911                          3   
1009  1009  0.0  1.0  0.0  0.0   536  1681                          3   
1010  1010  0.0  1.0  0.0  0.0   528  1710                          3   
1011  1011  0.0  1.0  0.0  0.0   454  1675                          3   
1012  1012  1.0  0.0  0.0  0.0   427  1637                          1   

      lym_density  epi_density  fib_density  inf_density  
0       23.675918    20.109769     0.000000       

In [3]:
# load data
loaders = []
for i in range(len(datasets)):
    loaders.append(DataLoader(dataset=datasets[i], batch_size=batch_size,
                    shuffle=True, collate_fn=datasets[i].collate_wrapper))
                    
loader = DataLoader(dataset=datasets[0], batch_size=batch_size,
                    shuffle=False, collate_fn=datasets[0].collate_wrapper)
for (idx, batch) in enumerate(loader):
    adj, features, edge_features, adj_relative_cos, edges, labels, dist = batch
    break

In [4]:
# load model
directory = "models/"
fname = "aagnn_c2a1_rev_drop03_epoch50_hid8_out1_70_30_split_exp_saved_model.pth"
path = os.path.join(directory, fname)
model = AAGNN(4, 8, 1, 0.3, 'cpu')
sigmoid = nn.Sigmoid()
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [14]:
# run model
t = 0.1
stats_per_batch = 5
results_dir = "results/70_30_split"

running_loss, total_loss = 0.0, 0.0
num_correct, num_examples = 0, 0
total_correct, total_examples, total_batches = 0, 0, 0
y_true, y_scores, y_pred = [], [], []
for i in range(len(datasets)):
    num_batches = int(ceil(len(datasets[i]) / batch_size))
    total_batches += num_batches
    # Added by Jorge
    edge_pred, neg_pred, classes, coords = [], [], None, None
    # --------------
    with torch.no_grad():
        for (idx, batch) in enumerate(loaders[i]):
            adj, features, densities, adj_relative_cos, edges, labels, dist = batch
            labels = labels.to(device)
        
            adj_relative_cos = {node: {(i, j): cos.to(device) for (i, j), cos in node_adj.items()} for node, node_adj in adj_relative_cos.items()}
            adj, features = adj.to(device), features.to(device)
            out = model(features, adj, adj_relative_cos)
                
            all_pairs = torch.mm(out, out.t())
            all_pairs = sigmoid(all_pairs)
            scores = 1 - all_pairs[edges.T]
            predictions = (scores >= t).long()

            num_correct += torch.sum(predictions == labels.long()).item()
            total_correct += torch.sum(predictions == labels.long()).item()
            num_examples += len(labels)
            total_examples += len(labels)
            y_true.extend(labels.detach().cpu().numpy())
            y_scores.extend(scores.detach().cpu().numpy())
            y_pred.extend(predictions.detach().cpu().numpy())
            if (idx + 1) % stats_per_batch == 0:
                accuracy = num_correct / num_examples
                if (torch.sum(labels.long() == 0).item() > 0) and (torch.sum(labels.long() == 1).item() > 0):
                    area = roc_auc_score(labels.detach().cpu().numpy(), scores.detach().cpu().numpy())
                    print('    ROC-AUC score: {:.4f}'.format(area))
                num_correct, num_examples = 0, 0
        num_correct, num_examples = 0, 0

total_accuracy = total_correct / total_examples
print('Threshold: {:.4f}, accuracy: {:.4f}'.format(t, total_correct / total_examples))
y_true = np.array(y_true).flatten()
y_scores = np.array(y_scores).flatten()
y_pred = np.array(y_pred).flatten()
report = classification_report(y_true, y_pred, digits=4)
area = roc_auc_score(y_true, y_scores)
print('ROC-AUC score: {:.4f}'.format(area))
print('Classification report\n', report)

    ROC-AUC score: 0.8710
    ROC-AUC score: 1.0000
    ROC-AUC score: 1.0000
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.7185
    ROC-AUC score: 0.8046
    ROC-AUC score: 0.9167
    ROC-AUC score: 0.8500
    ROC-AUC score: 0.9821
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.6500
    ROC-AUC score: 0.9833
    ROC-AUC score: 0.6500
    ROC-AUC score: 0.9852
    ROC-AUC score: 0.7926
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.8506
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.9333
    ROC-AUC score: 0.8387
    ROC-AUC score: 0.8000
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.8710
    ROC-AUC score: 0.8736
    ROC-AUC score: 0.9032
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.8000
    ROC-AUC score: 0.8833
    ROC-AUC score: 1.0000
    ROC-AUC score: 0.9000
    ROC-AUC score: 0.6407
    ROC-AUC score: 0.6897
    ROC-AUC score: 0.8125
    ROC-AUC score: 0.9885
    ROC-AUC score: 0.6452
    ROC-AUC score: 0.8667
    ROC-AUC score: 0.6322
    ROC-AUC score: 0.7667
    ROC-AUC 

In [25]:
features = torch.Tensor([
    [1, 2, 3, 4],
    [1, 1, 1, 1],
    [1, 2, 2, 1]
])

weight0 = torch.Tensor([
    [1, 1, 1, 1, 1],
    [1, 2, 3, 4, 5],
    [2, 2, 2, 2, 2],
    [5, 4, 3, 2, 1]
])

weight1 = torch.Tensor([
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0],
    [2, 2, 2, 2, 2],
    [0, 0, 0, 0, 0]
])

edge_features = torch.Tensor([
    [
        [1, 2, 3],
        [3, 2, 1],
        [2, 2, 2]
    ],
    [
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]
    ]
])

In [27]:
x0 = torch.matmul(features, weight0)
x1 = torch.matmul(features, weight1)
x2 = torch.matmul(edge_features, x0)
output = torch.cat([xi for xi in x2], dim=1)

In [28]:
output

tensor([[ 83.,  84.,  85.,  86.,  87.,  50.,  49.,  48.,  47.,  46.],
        [117., 112., 107., 102.,  97., 100.,  98.,  96.,  94.,  92.],
        [100.,  98.,  96.,  94.,  92., 150., 147., 144., 141., 138.]])

In [31]:
x0

tensor([[29., 27., 25., 23., 21.],
        [ 9.,  9.,  9.,  9.,  9.],
        [12., 13., 14., 15., 16.]])

In [29]:
x1

tensor([[7., 7., 7., 7., 7.],
        [3., 3., 3., 3., 3.],
        [5., 5., 5., 5., 5.]])

In [30]:
x2

tensor([[[ 83.,  84.,  85.,  86.,  87.],
         [117., 112., 107., 102.,  97.],
         [100.,  98.,  96.,  94.,  92.]],

        [[ 50.,  49.,  48.,  47.,  46.],
         [100.,  98.,  96.,  94.,  92.],
         [150., 147., 144., 141., 138.]]])

In [32]:
x2 + x1

tensor([[[ 90.,  91.,  92.,  93.,  94.],
         [120., 115., 110., 105., 100.],
         [105., 103., 101.,  99.,  97.]],

        [[ 57.,  56.,  55.,  54.,  53.],
         [103., 101.,  99.,  97.,  95.],
         [155., 152., 149., 146., 143.]]])

In [4]:
p = utils.parse_args()

usage: ipykernel_launcher.py [-h] [--json JSON]
                             [--stats_per_batch STATS_PER_BATCH]
                             [--results_dir RESULTS_DIR]
                             [--saved_models_dir SAVED_MODELS_DIR]
                             [--task {unsupervised,link_prediction}]
                             [--agg_class {<class 'layers.MeanAggregator'>,<class 'layers.LSTMAggregator'>,<class 'layers.MaxPoolAggregator'>,<class 'layers.MeanPoolAggregator'>}]
                             [--cuda] [--dropout DROPOUT]
                             [--hidden_dims [HIDDEN_DIMS ...]]
                             [--out_dim OUT_DIM] [--num_samples NUM_SAMPLES]
                             [--classifier {pos_sig,neg_sig,mlp}]
                             [--batch_size BATCH_SIZE] [--epochs EPOCHS]
                             [--lr LR] [--weight_decay WEIGHT_DECAY]
                             [--debug DEBUG] [--save] [--test] [--val]
ipykernel_launcher.py: error: unrecog

SystemExit: 2

/Users/hasegawatai/anaconda3/envs/DigitalPathology/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
a = torch.Tensor([
    [1, 2, 3],
    [2, 3, 4],
    [3, 4, 5]
])

In [8]:
a.reshape(-1, 3, 3)

tensor([[[1., 2., 3.],
         [2., 3., 4.],
         [3., 4., 5.]]])